calculate the normalized density of those inactivated disciplines

In [30]:
import pandas as pd
import numpy as np
from collections import Counter
import seaborn as sns
from itertools import product
import matplotlib.pyplot as plt

In [24]:
dpath='../../data/dropbox/Data/Derived/Publication_based/Transition/density_and_timeflag_full_1973-2017.csv'
density_norm='../../data/dropbox/Data/Derived/Publication_based/Transition/cntry_densitynormed_avg.csv'
pltfile = '../../data/dropbox/Figs/transition/full/USvsChina.pdf'

In [4]:
density_df=pd.read_csv(dpath)

In [5]:
def density_zscore(data):
    meta_normed=pd.DataFrame()
    cntrylist=data.COUNTRY.unique()
    yearlist=data.CRRT_TIME.unique()
    for c, y in product(cntrylist,yearlist):
        df=data[(data.COUNTRY==c)&(data.CRRT_TIME==y)]
        if not df.empty:
            df=df[df.st0==0]
            density_avg=np.average(df.Density.values)
            density_std=np.std(df.Density.values)
            df['Density_norm']=df['Density'].apply(lambda x: (x-density_avg)/(density_std))
            meta_normed=pd.concat([meta_normed,df])
    return meta_normed
density_norm=density_zscore(density_df)

In [18]:
density_norm.head()

,DIS,Density,st0,st1,COUNTRY,CRRT_TIME,Density_norm
0,Acoustics,0.129612,0,0,Afghanistan,1973-1977,-1.150536
2,Aerospace Technology,0.131653,0,0,Afghanistan,1973-1977,-1.094618
4,Allergy,0.188202,0,0,Afghanistan,1973-1977,0.454860
5,Analytical Chemistry,0.150759,0,0,Afghanistan,1973-1977,-0.571096
6,Anatomy & Morphology,0.179148,0,0,Afghanistan,1973-1977,0.206772


In [23]:
density_act=density_norm[density_norm.st1==1]
cntry_avg=density_act.groupby(['COUNTRY','CRRT_TIME'])['Density_norm'].mean().reset_index()
cntry_avg.head()

,COUNTRY,CRRT_TIME,Density_norm
0,Afghanistan,1973-1977,1.271279
1,Afghanistan,1978-1982,1.165540
2,Afghanistan,1983-1987,0.599289
3,Afghanistan,1988-1992,0.330956
4,Afghanistan,1993-1997,0.458757


In [25]:
cntry_avg.to_csv(density_norm,index=False)

In [27]:
top2=['United States','China']
top2_df=density_act[density_act['COUNTRY'].isin(top2)]
top2_df.head()

,DIS,Density,st0,st1,COUNTRY,CRRT_TIME,Density_norm
40050,Applied Mathematics,0.110708,0,1,China,1973-1977,-0.062458
40051,Applied Physics,0.076840,0,1,China,1973-1977,-1.018077
40088,Fluids & Plasmas,0.070549,0,1,China,1973-1977,-1.195582
40093,General Chemistry,0.083618,0,1,China,1973-1977,-0.826835
40095,General Mathematics,0.111498,0,1,China,1973-1977,-0.040167


In [ ]:
fig, ax=plt.subplots()
sns.kdeplot(data=top2_df, x='Density_norm',hue='COUNTRY',ax=ax)
plt.ylabel("Kernal Density Estimation",fontsize=15)
plt.xlabel("Normalized Disciplinary Density",fontsize=15)
plt.legend(labels=['United States','China'],frameon=False)
plt.savefig(pltfile)